# In this part, I will clean the data in Kansas Crime dataset.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pylab
import numpy as np
from matplotlib.mlab import griddata
import datetime
%matplotlib inline

In [2]:
kansas_data=pd.read_csv('crime.csv')

/Users/apple/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7,17,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
kansas_data.head()

,Unnamed: 0,id,agency,daterept,hourrept,ucrcode,offense,neighborhood,subdivision,spx,...,city,state,zip,theftdescription,inactivitydescription,premisedescription,incidentid,apid,crimetype,geom
0,1,1,JCSO,9/11/09 0:00,1120.0,90Z,FAILURE TO APPEAR,NaN,0.0,1423436.57,...,SALINA,KS,67401,NaN,Not Applicable,Government / Public Building,D09004321,NaN,Part 2,01010000205B0D00001F85EB914CB835413D0AD7A3461C...
1,2,2,JCSO,7/3/15 0:00,1514.0,35A,POSSESSION OF HALLUCINOGENIC DRUG,NaN,-1.0,2144629.87,...,EUDORA,KS,66025,NaN,Not Applicable,Highway,D15002447,NaN,Part 2,01010000205B0D0000F6285CEFBA5C4041CDCCCCCCF0B0...
2,3,3,JCSO,6/27/05 0:00,1346.0,90Z,DRIVING WHILE HABITUAL VIOLATOR,NaN,-1.0,2149483.00,...,COUNTY,KS,NaN,NaN,Not Applicable,Highway,D05002424,NaN,Part 2,01010000205B0D00000000008035664041EC51B81E61C5...
3,4,4,JCSO,2/5/05 0:00,145.0,90Z,FAILURE TO APPEAR,NaN,-1.0,2149483.00,...,COUNTY,KS,66018,NaN,Not Applicable,Highway,D05000725,NaN,Part 2,01010000205B0D00000000008035664041EC51B81E61C5...
4,5,5,JCSO,4/14/10 0:00,1952.0,90D,DRIVING UNDER INFLUENCE OF ALCOHOL OR DRUGS; U...,JC1,10200.0,2149561.34,...,EUDORA,KS,66025,NaN,Not Applicable,Highway,D10001671,NaN,Part 2,01010000205B0D0000B81E85AB5C6640410AD7A37055C5...


In [4]:
kansas_data=kansas_data.drop(kansas_data.columns[0],axis=1)

In [5]:
kansas_data['state'].value_counts()

KS    332773
MO        26
K          8
Q          1
62         1
WI         1
Name: state, dtype: int64

In [6]:
kansas_data.shape

(335614, 24)

## I want to find out how many null values are in each feature in our dataset.

In [7]:
def missing_count(dataset):
    missing={}
    for col in dataset.columns:
        missing[col]=len(dataset[dataset[col].isnull()])
    return missing

In [8]:
missing_count(kansas_data)

{'agency': 1,
 'apid': 97698,
 'city': 203,
 'crimetype': 0,
 'daterept': 0,
 'fulllocation': 0,
 'geom': 0,
 'hourrept': 1,
 'id': 0,
 'inactivitydescription': 34429,
 'incidentid': 0,
 'location': 0,
 'neighborhood': 70225,
 'offense': 1,
 'premisedescription': 30360,
 'spx': 0,
 'spy': 0,
 'state': 2804,
 'subdivision': 30870,
 'theftdescription': 235054,
 'ucrcode': 2709,
 'x': 0,
 'y': 0,
 'zip': 24798}

In [9]:
kansas_data[kansas_data.hourrept.isnull() == True]

,id,agency,daterept,hourrept,ucrcode,offense,neighborhood,subdivision,spx,spy,...,city,state,zip,theftdescription,inactivitydescription,premisedescription,incidentid,apid,crimetype,geom
23073,23003,OPD,10/28/04 0:00,NaN,90C,DISORDERLY CONDUCT,NaN,-1.0,2212392.0,215334.0,...,OLATHE,KS,NaN,NaN,Not Applicable,Parking Lot/ or Parking Garage,20040018460,2.000000e+17,Part 2,01010000205B0D00000000000014E14041000000003049...


## One way to fix the missing value problem of ucrcode is to find it from the offense names.

In [10]:
kansas_data[kansas_data.ucrcode.isnull() == True]

,id,agency,daterept,hourrept,ucrcode,offense,neighborhood,subdivision,spx,spy,...,city,state,zip,theftdescription,inactivitydescription,premisedescription,incidentid,apid,crimetype,geom
171465,171445,OPPD,2/26/07 0:00,906.0,NaN,INFO - ANIMAL CALL,21,3600.0,2252844.67,247308.99,...,OVERLAND_PARK,KS,66212,NaN,Not Applicable,NaN,2007004219,1.990000e+17,Part 2,01010000205B0D00005C8FC25516304141B81E85EB6730...
235261,235086,OPPD,1/3/14 0:00,1946.0,NaN,INFORMATION - TRUCK INSPECTION,2,705400.0,2259113.00,243467.00,...,OVERLAND_PARK,KS,66212,NaN,Not Applicable,NaN,2014000138,1.990000e+17,Part 2,01010000205B0D000000000080543C41410000000058B8...
332907,3000001,SPD,5/30/15 0:00,2138.0,NaN,"BATTERY, DOMESTIC; RUDE CONTACT",NaN,NaN,2245167.00,265618.00,...,SHAWNEE,KS,66203,NaN,NaN,Single Residence,CN1502414,2.010000e+17,Part 2,01010000205B0D00000000008017214141000000004836...
332908,3000002,SPD,8/11/15 0:00,1551.0,NaN,"BATTERY, DOMESTIC; BODILY HARM",NaN,NaN,2247918.00,256666.00,...,SHAWNEE,KS,66214,NaN,NaN,Apartment Complex,CN1503667,2.000000e+17,Part 2,01010000205B0D0000000000007726414100000000D054...
332909,3000003,SPD,7/16/15 0:00,1527.0,NaN,THEFT OF PROPERTY OR SERVICES; MISDEMEANOR,NaN,NaN,2233065.00,264462.00,...,SHAWNEE,KS,66216,All Other,NaN,Duplex / Individual Multiple Housing,CN1503247,2.010000e+17,Part 1,01010000205B0D00000000008074094141000000003824...
332910,3000004,SPD,10/24/15 0:00,1529.0,NaN,"CRIMINAL THREAT; CAUSE TERROR, EVACUTION OR DI...",NaN,NaN,2244977.00,258625.00,...,SHAWNEE,KS,66203,NaN,NaN,Single Residence,CN1504845,2.000000e+17,Part 2,01010000205B0D000000000080B8204141000000000892...
332911,3000005,SPD,6/24/14 0:00,1556.0,NaN,THEFT OF PROPERTY OR SERVICES; FELONY,NaN,NaN,2231941.00,259516.00,...,SHAWNEE,KS,66217,Theft from a Motor Vehicle,NaN,Apartment Complex,OR1400118,2.000000e+17,Part 1,01010000205B0D0000000000804207414100000000E0AD...
332912,3000006,SPD,6/1/15 0:00,834.0,NaN,CRIMINAL DAMAGE TO PROPERTY; MISDEMEANOR,NaN,NaN,2224779.00,261970.00,...,SHAWNEE,KS,66217,NaN,NaN,City Street,OR1500149,2.000000e+17,Part 2,01010000205B0D00000000008045F940410000000090FA...
332913,3000007,SPD,5/13/15 0:00,424.0,NaN,POSSESS CERTAIN HALLUCINOGENIC DRUGS (MARIJUANA),NaN,NaN,2244310.00,263657.00,...,SHAWNEE,KS,66203,NaN,NaN,Park Trails / Open Recreational Areas,CN1502126,2.010000e+17,Part 2,01010000205B0D0000000000006B1F414100000000A417...
332914,3000008,SPD,2/12/15 0:00,1116.0,NaN,PARAPHERNALIA - USE/POSSESS W/INTENT TO USE DR...,NaN,NaN,2226302.00,267735.00,...,SHAWNEE,KS,66217,NaN,NaN,Duplex / Individual Multiple Housing,CN1500670,2.010000e+17,Part 2,01010000205B0D0000000000003FFC4041000000005C57...


In [11]:
kansas_data[kansas_data.ucrcode.isnull() == True].offense.value_counts()

THEFT OF PROPERTY OR SERVICES; MISDEMEANOR                                          580
THEFT OF PROPERTY OR SERVICES; FELONY                                               376
CRIMINAL DAMAGE TO PROPERTY; MISDEMEANOR                                            194
IDENTITY THEFT; RECEIVE BENEFITS <$100K                                             131
POSSESS CERTAIN HALLUCINOGENIC DRUGS (MARIJUANA)                                    101
BURGLARY; ENTER DWELLING TO COMMIT FELONY, THEFT OR SEXUAL BATTERY                   89
BATTERY, DOMESTIC; BODILY HARM                                                       86
IDENTITY THEFT; ECONOMIC/BODILY HARM <$100K                                          82
BATTERY, SIMPLE                                                                      71
BATTERY, DOMESTIC; RUDE CONTACT                                                      70
THEFT OF PROPERTY OR SERVICES; VALUE $1,000 TO $25,000                               61
AGGRAVATED BURGLARY             

In [12]:
sum(kansas_data[kansas_data.ucrcode.isnull() == True].offense.value_counts())

2709

### We construct the offense list containing all the offense names corresponding to the cases where ucrcode is missing.

In [13]:
missing_offense_list = [item for item in set(kansas_data[kansas_data.ucrcode.isnull() == True].offense)]

In [14]:
missing_offense_list

['MISTREAT DEPENDANT ADULT; LOSS > $1,000 BUT < $25,000',
 'UNLAWFUL ABUSE OF TOXIC VAPORS',
 'DISTRIBUTE OPIATE/NARC/CERTAIN STIM/HEROIN; DOSE  < 10',
 'FORGERY; MAKING OR ALTERING A WRITTEN INSTRUMENT',
 'ELECTRONIC SOLICITATION; OFFENDER BELIEVES CHILD <14',
 'LITTERING - CITY ORDINANCE VIOLATION',
 'MIP',
 'AGGRAVATED ARSON; SUBSTANTIAL RISK TO GREAT BODILY HARM',
 'CRIMINAL DISCHARGE OF A FIREARM; OCCUPIED DWELLING NO BODILY HARM',
 'CRIMINAL DAMAGE TO PROPERTY <$1,000',
 'RAPE; SEXUAL INTERCOURSE WITHOUT CONSENT AND USE OF FORCE',
 'VIOLATION OF PROTECTION ORDER; ABUSE ORDER',
 'AGGRAVATED ROBBERY',
 'DISTRIBUTE MARIJUANA;  < 25 G',
 'VIOLATION OF PROTECTION ORDER; STALKING ORDER',
 'THEFT OF PROPERTY OR SERVICES; FELONY',
 'CRIMINAL USE OF A FINANCIAL CARD; FELONY',
 'HARRASS BY TELCOM DEVICE; REPEAT PHONE CALLS OR FAX',
 'INTERFERENCE WITH A LAW ENFORCEMENT OFFICER; FALSELY REPORTING A CRIME',
 'WINDOW PEEPING PROHIBITED',
 'CRIMINAL POSS OF FIREARM BY FELON; FELONY CONV < 5YRS

## Take a look at the 7th offense in the missing_offense_list

In [15]:
kansas_data[kansas_data['offense'] == missing_offense_list[7]]['ucrcode']

2473      200
46460     200
172692    200
173934    200
214924    200
272685    200
334210    NaN
Name: ucrcode, dtype: object

In [16]:
kansas_data[kansas_data['offense'] == missing_offense_list[7]]['ucrcode'].value_counts()

200    6
Name: ucrcode, dtype: int64

In [17]:
kansas_data[kansas_data['offense'] == missing_offense_list[7]]['ucrcode'].value_counts()[:1].index[0]

'200'

In [18]:
kansas_data[kansas_data['offense'] == missing_offense_list[7]]['ucrcode'].isnull()

2473      False
46460     False
172692    False
173934    False
214924    False
272685    False
334210     True
Name: ucrcode, dtype: bool

In [19]:
len(kansas_data[kansas_data['offense'] == missing_offense_list[7]]['ucrcode'][kansas_data[kansas_data['offense'] == missing_offense_list[7]]['ucrcode'].isnull() == False])

6

In [20]:
#kansas_data[kansas_data.ucrcode.isnull() == True][kansas_data['offense'] == missing_offense_list[7]]['ucrcode']= kansas_data[kansas_data['offense'] == missing_offense_list[7]]['ucrcode'].value_counts()[:1].index[0]     

In [21]:
#kansas_data[kansas_data.ucrcode.isnull() == True][kansas_data['offense'] == missing_offense_list[7]]

In [22]:
index_7 = kansas_data[kansas_data.ucrcode.isnull() == True][kansas_data['offense'] == missing_offense_list[7]].index

/Users/apple/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [23]:
index_7

Int64Index([334210], dtype='int64')

In [24]:
majority_category_7 =kansas_data[kansas_data['offense'] == missing_offense_list[7]]['ucrcode'].value_counts()[:1].index[0]  
majority_category_7

'200'

In [25]:
#kansas_data.iloc[index_7]['ucrcode'] = kansas_data[kansas_data['offense'] == missing_offense_list[7]]['ucrcode'].value_counts()[:1].index[0]     

In [26]:
#kansas_data.loc[index_7]['ucrcode']  = majority_category_7

In [27]:
#kansas_data.replace(kansas_data.loc[index_7]['ucrcode'], majority_category_7, inplace = True)

In [28]:
kansas_data.loc[index_7]['ucrcode']

334210    NaN
Name: ucrcode, dtype: object

In [29]:
#kansas_data.iloc[index_7] = kansas_data[kansas_data['offense'] == missing_offense_list[7]]['ucrcode'].value_counts()[:1].index[0]     

In [30]:
#kansas_data.loc[index_7]['ucrcode']

In [31]:
kansas_data[kansas_data['offense'] == missing_offense_list[7]]['ucrcode'].value_counts()[:1].index[0]     

'200'

In [32]:
kansas_data.loc[index_7,'ucrcode'] = majority_category_7

In [33]:
kansas_data.iloc[index_7]

,id,agency,daterept,hourrept,ucrcode,offense,neighborhood,subdivision,spx,spy,...,city,state,zip,theftdescription,inactivitydescription,premisedescription,incidentid,apid,crimetype,geom
334210,3001302,SPD,10/12/15 0:00,1316.0,200,AGGRAVATED ARSON; SUBSTANTIAL RISK TO GREAT BO...,NaN,NaN,2247631.0,256179.0,...,SHAWNEE,KS,66214,NaN,NaN,Apartment Complex,CN1504060,2.000000e+17,Part 1,01010000205B0D000000000080E7254141000000009845...


In [34]:
kansas_data.iloc[index_7]['ucrcode']

334210    200
Name: ucrcode, dtype: object

### This is the first key function to fill nans of ucrcode!

In [35]:
missing_offense_list_2 = []
for i in range(len(missing_offense_list)):
    if len(kansas_data[kansas_data['offense'] == missing_offense_list[i]]['ucrcode'][kansas_data[kansas_data['offense'] == missing_offense_list[i]]['ucrcode'].isnull() == False]) >= 1:
        index_i = kansas_data[kansas_data.ucrcode.isnull() == True][kansas_data['offense'] == missing_offense_list[i]].index
        kansas_data.loc[index_i,'ucrcode'] = kansas_data[kansas_data['offense'] == missing_offense_list[i]]['ucrcode'].value_counts()[:1].index[0]     
    else:
        missing_offense_list_2.append(missing_offense_list[i])   

/Users/apple/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [36]:
sum(kansas_data[kansas_data.ucrcode.isnull() == True].offense.value_counts())

1027

### In this case, we can assign an ucrcode to this type of offense without changing the name of the offense.

## Now let's take a look at the 0th offense in the missing_offense_list

In [37]:
kansas_data[kansas_data['offense'] == missing_offense_list[0]]['ucrcode']

333847    NaN
Name: ucrcode, dtype: object

In [38]:
kansas_data[kansas_data['offense'] == missing_offense_list[0]]['ucrcode'].isnull()

333847    True
Name: ucrcode, dtype: bool

In [39]:
missing_offense_list[0].split(';')[0]

'MISTREAT DEPENDANT ADULT'

In [40]:
index_0 = kansas_data.offense[kansas_data['offense'].str.contains(missing_offense_list[0].split(';')[0]) == True].index

In [41]:
kansas_data.iloc[index_0]['ucrcode']   #333847

3152      26A
8196      13B
17055     26A
50978     26A
50979     26A
67977     26A
69926     13B
81745     26A
81746     26A
84068     26A
86498     26A
86518     26A
86602     13B
86717     26A
87318     26A
87695     26A
91654     26A
100965    26A
101039    26A
104599    26A
127514    26A
134889    26A
161628    26A
162977    26A
167631    26A
167632    26A
167642    26A
167644    26A
179403    26A
191197    26A
191689    26A
192176    13B
194606    26A
198489    26A
206514    26A
209190    90J
209263    26A
211493    13B
224468    26A
228741    26A
247766    26A
249599    26A
251035    26A
254452    26A
265291    26A
265420    26A
273446    26A
273449    26A
275121    26A
278235    26A
282346    26A
295712    26A
297533    26A
299714    13B
301302    26A
301719    13B
333847    NaN
Name: ucrcode, dtype: object

In [42]:
kansas_data.iloc[index_0][kansas_data.ucrcode.isnull() == True].index

/Users/apple/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


Int64Index([333847], dtype='int64')

In [43]:
index_missing_list_0 = kansas_data.offense[kansas_data['offense'].str.contains(missing_offense_list[0].split(';')[0]) == True].index

In [44]:
len(kansas_data.offense[kansas_data['offense'].str.contains(missing_offense_list[0].split(';')[0]) == True][kansas_data.offense[kansas_data['offense'].str.contains(missing_offense_list[0].split(';')[0]) == True].isnull() == False]) 

57

In [45]:
kansas_data.iloc[index_missing_list_0]['ucrcode'].value_counts()

26A    48
13B     7
90J     1
Name: ucrcode, dtype: int64

In [46]:
kansas_data.iloc[index_missing_list_0]['ucrcode'].value_counts()[:1]

26A    48
Name: ucrcode, dtype: int64

In [47]:
kansas_data.iloc[index_missing_list_0]['ucrcode'].value_counts()[:1].index[0]

'26A'

### In this case, we can split the offense name, and take the importance piece , find the ucrcode that corresponds to the most number of important piece 

## Take a look at the 2nd offense in the missing_offense_list

In [48]:
kansas_data[kansas_data['offense'] == missing_offense_list[2]]['ucrcode']

333943    NaN
335526    NaN
Name: ucrcode, dtype: object

In [49]:
missing_offense_list[2].split(';')[0]

'DISTRIBUTE OPIATE/NARC/CERTAIN STIM/HEROIN'

In [50]:
kansas_data.offense[kansas_data['offense'].str.contains(missing_offense_list[2].split(';')[0]) == True]

272107    DISTRIBUTE OPIATE/NARC/CERTAIN STIM/HEROIN; DO...
273107    DISTRIBUTE OPIATE/NARC/CERTAIN STIM/HEROIN; DO...
333943    DISTRIBUTE OPIATE/NARC/CERTAIN STIM/HEROIN; DO...
335526    DISTRIBUTE OPIATE/NARC/CERTAIN STIM/HEROIN; DO...
Name: offense, dtype: object

In [51]:
index_missing_list_2 = kansas_data.offense[kansas_data['offense'].str.contains(missing_offense_list[2].split(';')[0]) == True].index

In [52]:
len(kansas_data.iloc[index_missing_list_2]['ucrcode'][kansas_data.iloc[index_missing_list_2]['ucrcode'].isnull() == False])

2

In [53]:
kansas_data.iloc[index_missing_list_2]['ucrcode'].value_counts()

35A    2
Name: ucrcode, dtype: int64

In [54]:
kansas_data.iloc[index_missing_list_2]['ucrcode'].value_counts()[:1].index[0]

'35A'

### The case for 2nd offense is similar to the case for 7th offense, we can pick the important piece from the offense name and find its ucrcode.

## Take a look at the fifth offense in the missing_offense_list

In [55]:
kansas_data[kansas_data['offense'] == missing_offense_list[5]]['ucrcode']

333644    NaN
335015    NaN
335477    NaN
Name: ucrcode, dtype: object

In [56]:
missing_offense_list[5]

'LITTERING - CITY ORDINANCE VIOLATION'

In [57]:
missing_offense_list[5].split(';')[0]

'LITTERING - CITY ORDINANCE VIOLATION'

In [58]:
kansas_data.offense[kansas_data['offense'].str.contains(missing_offense_list[5].split(';')[0]) == True]

333644    LITTERING - CITY ORDINANCE VIOLATION
335015    LITTERING - CITY ORDINANCE VIOLATION
335477    LITTERING - CITY ORDINANCE VIOLATION
Name: offense, dtype: object

In [59]:
index_missing_list_5 = kansas_data.offense[kansas_data['offense'].str.contains(missing_offense_list[5].split(';')[0]) == True].index

In [60]:
len(kansas_data.iloc[index_missing_list_5]['ucrcode'][kansas_data.iloc[index_missing_list_5]['ucrcode'].isnull() == False])

0

In [61]:
len(kansas_data.iloc[index_missing_list_5]['ucrcode'][kansas_data.iloc[index_missing_list_5]['ucrcode'].isnull() == True])

3

In [62]:
kansas_data.iloc[index_missing_list_5]['ucrcode'].isnull()

333644    True
335015    True
335477    True
Name: ucrcode, dtype: bool

In [63]:
kansas_data.iloc[index_missing_list_5]['ucrcode'].value_counts()

Series([], Name: ucrcode, dtype: int64)

### In this case, we just cannot find a ucrcode for this type of offense.

## Great! Now we have found a way to asign missing ucrcode to ambiguous offense code!

## This is a key function to assign ucrcode to missing items using offense description!

In [64]:
len(kansas_data.iloc[index_missing_list_5]['ucrcode'][kansas_data.iloc[index_missing_list_5]['ucrcode'].isnull() == False])

0

In [65]:
missing_offense_list_2 = []
for i in range(len(missing_offense_list)):
    if len(kansas_data[kansas_data['offense'] == missing_offense_list[i]]['ucrcode'][kansas_data[kansas_data['offense'] == missing_offense_list[i]]['ucrcode'].isnull() == False]) >= 1:
        index_i = kansas_data[kansas_data.ucrcode.isnull() == True][kansas_data['offense'] == missing_offense_list[i]].index
        kansas_data.loc[index_i,'ucrcode'] = kansas_data[kansas_data['offense'] == missing_offense_list[i]]['ucrcode'].value_counts()[:1].index[0]     
    else:
        missing_offense_list_2.append(missing_offense_list[i])   

/Users/apple/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [66]:
index_0_missing = kansas_data.iloc[index_0][kansas_data.ucrcode.isnull() == True].index

/Users/apple/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [67]:
kansas_data.loc[index_0_missing, 'ucrcode']

333847    NaN
Name: ucrcode, dtype: object

In [68]:
final_offense_list = []
for i in range(len(missing_offense_list_2)):
       index_i = kansas_data.offense[kansas_data['offense'].str.contains(missing_offense_list_2[i].split(';')[0]) == True].index
       if len(kansas_data.iloc[index_i]['ucrcode'][kansas_data.iloc[index_i]['ucrcode'].isnull() == False]) > 0:
             index_i_missing = kansas_data.iloc[index_i][kansas_data.ucrcode.isnull() == True].index
             kansas_data.loc[index_i_missing,'ucrcode'] = kansas_data.iloc[index_i]['ucrcode'].value_counts()[:1].index[0]  
       else:
            index_ii = kansas_data.offense[kansas_data['offense'].str.contains(missing_offense_list_2[i].split(',')[0]) == True].index
            if len(kansas_data.iloc[index_ii]['ucrcode'][kansas_data.iloc[index_ii]['ucrcode'].isnull() == False]) > 0:
                index_ii_missing = kansas_data.iloc[index_ii][kansas_data.ucrcode.isnull() == True].index
                kansas_data.loc[index_ii_missing, 'ucrcode'] = kansas_data.iloc[index_ii]['ucrcode'].value_counts()[:1].index[0]

/Users/apple/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/Users/apple/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/apple/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:8: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
/Users/apple/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


In [69]:
sum(kansas_data[kansas_data.ucrcode.isnull() == True].offense.value_counts())

259

In [70]:
kansas_data[kansas_data.ucrcode.isnull() == True].offense.value_counts()

POSSESS CERTAIN HALLUCINOGENIC DRUGS (MARIJUANA)                                  101
HARRASS BY TELCOM DEVICE; REPEAT PHONE CALLS OR FAX                                41
HARRASS BY TELECOM DEVICE; THREATENING PHONE CALL OR FAX                           33
PARAPHERNALIA - USE/POSSESS W/INTENT TO USE DRUG PARAPHERNALIA INTO HUMAN BODY     33
DUI - ALCOHOL OR DRUGS; UNK SEVERITY                                                8
DELETE - ENTERED IN ERROR                                                           7
METH - POSSESS OPIATES/OPIUM/NARC DRUG AND CERTAIN STIMULANTS                       7
THEFT (POSS OF STOLEN PROPERTY); VALUE LESS THAN $1,000                             4
THEFT OR PROP/SERVICES; FIREARM < $25000                                            3
KANSAS CODE; CARE OF CHILDREN - DEFINITIONS                                         3
LITTERING - CITY ORDINANCE VIOLATION                                                3
FLEEING OR ATTEMPTING TO ELUDE A LEO; UNKNOWN SEVERITY

In [71]:
index_info_missing = kansas_data[kansas_data.offense == 'INFO - ANIMAL CALL'].index
index_info_missing

Int64Index([171465], dtype='int64')

In [72]:
kansas_data.loc[index_info_missing, 'ucrcode']

171465    NaN
Name: ucrcode, dtype: object

### I just consider the first four categories.

In [73]:
index_drug = kansas_data.offense[kansas_data['offense'].str.contains('POSSESS','HALLUCINOGENIC') == True].index

In [74]:
kansas_data.iloc[index_drug]['ucrcode'].value_counts()

35A    8007
35B    1628
280    1229
370     113
90Z     105
250     104
520      74
90G      24
23        4
90D       2
26D       1
26A       1
240       1
Name: ucrcode, dtype: int64

In [75]:
kansas_data.iloc[index_drug]['ucrcode'].value_counts()[:1].index[0]

'35A'

In [76]:
index_drug_missing = kansas_data[kansas_data.offense == 'POSSESS CERTAIN HALLUCINOGENIC DRUGS (MARIJUANA)'].index
index_drug_missing

Int64Index([332913, 332959, 332986, 333005, 333007, 333014, 333043, 333149,
            333258, 333273,
            ...
            335372, 335379, 335384, 335389, 335435, 335511, 335517, 335528,
            335537, 335597],
           dtype='int64', length=101)

In [77]:
kansas_data.loc[index_drug_missing, 'ucrcode'] = kansas_data.iloc[index_drug]['ucrcode'].value_counts()[:1].index[0]

In [78]:
kansas_data.loc[index_drug_missing, 'ucrcode'].head()

332913    35A
332959    35A
332986    35A
333005    35A
333007    35A
Name: ucrcode, dtype: object

### Next we find that there's no ucrcode for tele harrassment.

In [79]:
index_tele_harrass = kansas_data.offense[kansas_data['offense'].str.contains('HARRASS','DEVICE') == True].index

In [80]:
kansas_data.iloc[index_tele_harrass]['ucrcode'].value_counts()

Series([], Name: ucrcode, dtype: int64)

In [81]:
index_para = kansas_data.offense[kansas_data['offense'].str.contains('PARAPHERNALIA') == True].index

In [82]:
kansas_data.iloc[index_para]['ucrcode'].value_counts()[:1].index[0]

'35B'

In [83]:
index_para_missing = kansas_data[kansas_data.offense == 'PARAPHERNALIA - USE/POSSESS W/INTENT TO USE DRUG PARAPHERNALIA INTO HUMAN BODY'].index
index_para_missing

Int64Index([332914, 332943, 332973, 333247, 333304, 333348, 333399, 333494,
            333521, 333741, 333776, 333958, 334033, 334042, 334067, 334116,
            334175, 334195, 334236, 334273, 334396, 334506, 334529, 334531,
            334614, 335040, 335061, 335093, 335312, 335317, 335326, 335392,
            335490],
           dtype='int64')

In [84]:
kansas_data.loc[index_para_missing, 'ucrcode'] = kansas_data.iloc[index_para]['ucrcode'].value_counts()[:1].index[0]

In [85]:
kansas_data.loc[index_para_missing, 'ucrcode'].head()

332914    35B
332943    35B
332973    35B
333247    35B
333304    35B
Name: ucrcode, dtype: object

In [86]:
sum(kansas_data[kansas_data.ucrcode.isnull() == True].offense.value_counts())

125

In [87]:
kansas_data.ucrcode.value_counts()

23     90228
13B    35898
90Z    35552
290    29843
35A    19112
90D    19098
220    15310
13C    12794
23H    10335
240     9776
90C     6769
13A     5894
250     5279
90G     4491
26A     4366
26B     4090
35B     3278
26C     3031
90J     2859
11D     2348
90I     2127
11A     1633
280     1544
120     1522
100     1439
90A     1269
520     1114
200      925
370      697
90F      623
40B      513
40A      467
270      420
11B      418
36B      128
90E      114
09A       85
210       53
36A       13
09B       12
39A       10
510        5
26D        3
39C        2
09C        1
39B        1
Name: ucrcode, dtype: int64

In [88]:
#kansas_data[kansas_data.ucrcode == '39B']['offense']

In [89]:
#kansas_data[kansas_data.ucrcode == '39C']['offense']

### I want to find out the offense type that has very few incidents.

In [90]:
kansas_data[kansas_data.ucrcode == '09A']['offense'].head()

3820                MURDER IN THE SECOND DEGREE; RECKLESS
5396       MURDER IN THE 1ST DEGREE; UNKNOWN CIRCUMSTANCE
6286                           MURDER IN THE FIRST DEGREE
6568       MURDER IN THE 1ST DEGREE; UNKNOWN CIRCUMSTANCE
8416    MURDER IN THE 1ST DEGREE; INTENTIONAL AND PREM...
Name: offense, dtype: object

In [91]:
index_murder = kansas_data.offense[kansas_data['offense'].str.contains('MURDER') == True].index

In [92]:
index_murder

Int64Index([  3820,   5396,   6286,   6568,   8416,  10180,  10918,  15102,
             18445,  20285,
            ...
            319819, 321227, 322146, 322768, 323372, 323397, 326260, 330794,
            332082, 332979],
           dtype='int64', length=125)

In [93]:
kansas_data.iloc[index_murder].offense.head()

3820                MURDER IN THE SECOND DEGREE; RECKLESS
5396       MURDER IN THE 1ST DEGREE; UNKNOWN CIRCUMSTANCE
6286                           MURDER IN THE FIRST DEGREE
6568       MURDER IN THE 1ST DEGREE; UNKNOWN CIRCUMSTANCE
8416    MURDER IN THE 1ST DEGREE; INTENTIONAL AND PREM...
Name: offense, dtype: object

In [94]:
kansas_data.iloc[index_murder].ucrcode.value_counts()

09A    82
13A    39
90Z     4
Name: ucrcode, dtype: int64

In [95]:
kansas_data[kansas_data['ucrcode'] == '13A'].head()

,id,agency,daterept,hourrept,ucrcode,offense,neighborhood,subdivision,spx,spy,...,city,state,zip,theftdescription,inactivitydescription,premisedescription,incidentid,apid,crimetype,geom
67,68,JCSO,12/25/12 0:00,1124.0,13A,AGGRAVATED ASSAULT,JC1,521200.0,2152110.00,229272.00,...,EUDORA,KS,66025,NaN,Domestic Violence,Single Residence (includes attached garage),D12005562,2.000000e+17,Part 1,01010000205B0D000000000000576B404100000000C0FC...
126,127,JCSO,4/26/13 0:00,2320.0,13A,AGGRAVATED ASSAULT OF LEO,JCSO,-1.0,2148528.68,235565.64,...,EUDORA,KS,66025,NaN,Not Applicable,City Street,D13001774,NaN,Part 1,01010000205B0D0000713D0A5758644041EC51B81E6DC1...
182,183,JCSO,11/8/15 0:00,844.0,13A,AGGRAVATED ASSAULT,JC2,526000.0,2154923.00,204294.00,...,EUDORA,KS,66025,NaN,Domestic Violence,Single Residence (includes attached garage),D15004171,2.000000e+17,Part 1,01010000205B0D000000000080D57040410000000030F0...
199,200,JCSO,9/27/15 0:00,1327.0,13A,AGGRAVATED ASSAULT; WITH INTENT TO COMMIT ANY ...,JC2,526400.0,2159051.00,201396.00,...,EUDORA,KS,66025,NaN,Domestic Violence,Single Residence (includes attached garage),D15003626,2.000000e+17,Part 1,01010000205B0D000000000080E578404100000000A095...
275,276,JCSO,11/25/08 0:00,815.0,13A,AGGRAVATED ASSAULT,JC1,10400.0,2159222.00,247127.00,...,DESOTO,KS,66018,NaN,Not Applicable,Single Residence (includes attached garage),D08005203,2.000000e+17,Part 1,01010000205B0D0000000000003B79404100000000B82A...


In [96]:
index_blackmail = kansas_data.offense[kansas_data['offense'].str.contains('BLACKMAIL') == True].index

In [97]:
index_blackmail

Int64Index([  2842,   8596,   9365,  14182,  17155,  23254,  26347,  31838,
             48977,  53450,  61484,  63972,  76649,  90235,  96777,  98081,
             99617, 107360, 134332, 143101, 143681, 146127, 158579, 163665,
            165349, 166904, 169913, 186720, 192303, 192596, 199709, 201830,
            205913, 206901, 210308, 211199, 214194, 215411, 225197, 252828,
            256352, 259428, 275437, 280569, 282170, 298471, 299183, 305933,
            331849, 332469, 333612],
           dtype='int64')

In [98]:
#kansas_data[kansas_data.ucrcode == '26D']['offense']

In [99]:
#kansas_data[kansas_data.ucrcode == '510']['offense']

In [100]:
kansas_data[kansas_data.ucrcode == '39A']['offense']

14107     KNOWINGLY MAKE FALSE WRITING OR ENTRY AT TRACK
43238     KNOWINGLY MAKE FALSE WRITING OR ENTRY AT TRACK
64827     KNOWINGLY MAKE FALSE WRITING OR ENTRY AT TRACK
64894     KNOWINGLY MAKE FALSE WRITING OR ENTRY AT TRACK
64973     KNOWINGLY MAKE FALSE WRITING OR ENTRY AT TRACK
65893     KNOWINGLY MAKE FALSE WRITING OR ENTRY AT TRACK
65932     KNOWINGLY MAKE FALSE WRITING OR ENTRY AT TRACK
78124     KNOWINGLY MAKE FALSE WRITING OR ENTRY AT TRACK
83241     KNOWINGLY MAKE FALSE WRITING OR ENTRY AT TRACK
140545    KNOWINGLY MAKE FALSE WRITING OR ENTRY AT TRACK
Name: offense, dtype: object

In [101]:
kansas_data[kansas_data.ucrcode == '09B']['offense'].head()

9095               INVOLUNTARY MANSLAUGHTER; RECKLESSLY
9260                           INVOLUNTARY MANSLAUGHTER
33850                          INVOLUNTARY MANSLAUGHTER
51363    INVOLUNTARY MANSLAUGHTER; UNKNOWN CIRCUMSTANCE
92592                          INVOLUNTARY MANSLAUGHTER
Name: offense, dtype: object

In [102]:
kansas_data[kansas_data.ucrcode == '36A']['offense'].head()

11464     AGGRAVATED INCEST; INTERCOURSE/SODOMY W/RELATI...
64014     AGGRAVATED INCEST; INTERCOURSE/SODOMY W/RELATI...
88813                                                INCEST
107392      UNLAWFUL SEXUAL RELATIONS; UNKNOWN RELATIONSHIP
108415               AGGRAVATED INCEST; SUB-SECTION UNKNOWN
Name: offense, dtype: object

In [103]:
#kansas_data[kansas_data.ucrcode == '39A']['offense']

In [104]:
kansas_data[kansas_data.ucrcode == '210']['offense'].head()

2842     BLACKMAIL; DISSEMINATE IMAGE/PHOTO/VIDEO PER K...
8596                                             BLACKMAIL
9365         BLACKMAIL; STATEMENT TO CAUSE PUBLIC RIDECULE
14182                                            BLACKMAIL
17155                                            BLACKMAIL
Name: offense, dtype: object

In [105]:
#kansas_data[kansas_data.ucrcode == '90E']['offense'].head()

In [106]:
#kansas_data.to_csv('kansas_data_cleaned_ucrcode.csv', index = False)

### Data Manipulation of the time and date 

In [107]:
kansas_data['hourrept'] = kansas_data.hourrept.apply(lambda x: str(x).zfill(6))

In [108]:
kansas_data['hourrept'].head()

0    1120.0
1    1514.0
2    1346.0
3    0145.0
4    1952.0
Name: hourrept, dtype: object

In [109]:
kansas_data['hourrept'] = kansas_data.hourrept.apply(lambda x: x[:2]+':'+x[2:4]+':00')

In [110]:
kansas_data['hourrept'].head()

0    11:20:00
1    15:14:00
2    13:46:00
3    01:45:00
4    19:52:00
Name: hourrept, dtype: object

In [111]:
missing_count(kansas_data)['hourrept']

0

In [112]:
kansas_data['hourrept'][0].split(':')

['11', '20', '00']

In [113]:
kansas_data['Hour'] = kansas_data.hourrept.apply(lambda x: int(x.split(':')[0]))

In [114]:
kansas_data['Minute'] = kansas_data.hourrept.apply(lambda x: x.split(':')[1])

In [115]:
kansas_data['Minute'].value_counts()

00    23998
30    14601
45     8515
15     8087
20     6883
50     6795
40     6609
10     6548
05     5958
35     5875
55     5785
25     5691
01     5275
46     4911
06     4894
47     4894
58     4889
33     4889
38     4888
18     4883
21     4879
07     4871
14     4866
22     4854
27     4851
08     4850
24     4844
13     4841
31     4838
43     4838
      ...  
48     4829
44     4829
42     4818
12     4812
03     4806
23     4805
56     4805
34     4773
59     4762
49     4754
51     4752
37     4751
54     4742
36     4727
41     4726
02     4725
26     4723
16     4719
11     4717
29     4715
09     4706
28     4699
52     4695
19     4690
04     4678
17     4662
53     4658
39     4656
57     4650
0n        1
Name: Minute, Length: 61, dtype: int64

### We see there are a value '0n' in 'Minute' feature which is not what we want, so we need to remove this value.

In [116]:
kansas_data = kansas_data[kansas_data['Minute'] != '0n']

In [117]:
kansas_data['Minute'] = kansas_data.Minute.apply(lambda x: int(x))

In [118]:
kansas_data['Time'] = 60*kansas_data['Hour'] + kansas_data['Minute']

In [119]:
kansas_data['Time'].head()

0     680
1     914
2     826
3     105
4    1192
Name: Time, dtype: int64

In [120]:
kansas_data['Date'] = kansas_data.daterept.apply(lambda x: pd.to_datetime(x.split(' ')[0]))

In [121]:
kansas_data['Date'].head()

0   2009-09-11
1   2015-07-03
2   2005-06-27
3   2005-02-05
4   2010-04-14
Name: Date, dtype: datetime64[ns]

In [122]:
kansas_data['Day']=kansas_data['Date'].dt.day
kansas_data['DayOfWeek']=kansas_data['Date'].dt.dayofweek
kansas_data['Week']=kansas_data['Date'].dt.weekofyear
kansas_data['Month']=kansas_data['Date'].dt.month
kansas_data['Year']=kansas_data['Date'].dt.year

In [123]:
kansas_data['Day'].head()

0    11
1     3
2    27
3     5
4    14
Name: Day, dtype: int64

In [124]:
kansas_data['DayOfWeek'].head()

0    4
1    4
2    0
3    5
4    2
Name: DayOfWeek, dtype: int64

In [125]:
kansas_data['Week'].head()

0    37
1    27
2    26
3     5
4    15
Name: Week, dtype: int64

In [126]:
kansas_data['Month'].head()

0    9
1    7
2    6
3    2
4    4
Name: Month, dtype: int64

In [127]:
kansas_data['Year'].head()

0    2009
1    2015
2    2005
3    2005
4    2010
Name: Year, dtype: int64

In [128]:
month_names={1:"Jan",2:"Feb",3:"Mar",4:"Apr",5:"May",6:"Jun",7:"Jul",8:"Aug",9:"Sep",10:"Oct",11:"Nov",12:"Dec"}

In [129]:
day_names = {0: 'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday',
             4:'Friday', 5:'Saturday', 6:'Sunday'}

In [130]:
kansas_data['FullDayOfWeek'] = kansas_data['DayOfWeek'].apply(lambda x: day_names[x])
kansas_data['FullMonth'] = kansas_data['Month'].apply(lambda x: month_names[x])

In [131]:
kansas_data['FullMonth'].head()

0    Sep
1    Jul
2    Jun
3    Feb
4    Apr
Name: FullMonth, dtype: object

In [132]:
missing_count(kansas_data)

{'Date': 0,
 'Day': 0,
 'DayOfWeek': 0,
 'FullDayOfWeek': 0,
 'FullMonth': 0,
 'Hour': 0,
 'Minute': 0,
 'Month': 0,
 'Time': 0,
 'Week': 0,
 'Year': 0,
 'agency': 1,
 'apid': 97698,
 'city': 203,
 'crimetype': 0,
 'daterept': 0,
 'fulllocation': 0,
 'geom': 0,
 'hourrept': 0,
 'id': 0,
 'inactivitydescription': 34429,
 'incidentid': 0,
 'location': 0,
 'neighborhood': 70224,
 'offense': 1,
 'premisedescription': 30360,
 'spx': 0,
 'spy': 0,
 'state': 2804,
 'subdivision': 30870,
 'theftdescription': 235053,
 'ucrcode': 125,
 'x': 0,
 'y': 0,
 'zip': 24797}

In [133]:
#kansas_data.to_csv('kansas_data_timeAnducrcode_cleaned.csv')

## Next we clean the city names.

In [134]:
kansas_data.shape

(335613, 35)

In [135]:
kansas_data['city'].fillna('Unknown',inplace = True)

In [136]:
missing_count(kansas_data)['city']

0

In [137]:
kansas_data.loc[kansas_data['city'].str.contains('OVER'),'city'] = 'OVERLAND PARK'
kansas_data.loc[kansas_data['city'].str.contains('OVPKS'),'city'] = 'OVERLAND PARK'

kansas_data.loc[kansas_data['city'].str.contains('OLA'),'city'] = 'OLATHE'
kansas_data.loc[kansas_data['city'].str.contains('WEST WOOD'),'city'] = 'WESTWOOD'
kansas_data.loc[kansas_data['city'].str.contains('MSSION'),'city'] = 'MISSION'
kansas_data.loc[kansas_data['city'].str.contains('LAKE_QUIVIRA'),'city'] = 'LAKE QUIVIRA'
kansas_data.loc[kansas_data['city'].str.contains('OLA'),'city'] = 'OLATHE'
kansas_data.loc[kansas_data['city'].str.contains('OALTHE'),'city'] = 'OLATHE'
kansas_data.loc[kansas_data['city'].str.contains('OVER'),'city'] = 'OVERLAND PARK'
kansas_data.loc[kansas_data['city'].str.contains('OLA'),'city'] = 'OLATHE'
kansas_data.loc[kansas_data['city'].str.contains('SPRING_HILL'),'city'] = 'SPRING HILL'
kansas_data.loc[kansas_data['city'].str.contains('BONNER_SPRINGS'),'city'] = 'BONNER SPRING'
kansas_data.loc[kansas_data['city'].str.contains('OVELRAND'),'city'] = 'OVERLAND PARK'
kansas_data.loc[kansas_data['city'].str.contains('MISSION_WOODS'),'city'] = 'MISSION WOODS'
kansas_data.loc[kansas_data['city'].str.contains('SPRINGHILL'),'city'] = 'SPRING HILL'
kansas_data.loc[kansas_data['city'].str.contains('OATHE'),'city'] = 'OLATHE'
kansas_data.loc[kansas_data['city'].str.contains('OLTH'),'city'] = 'OLATHE'
kansas_data.loc[kansas_data['city'].str.contains('MISSIION'),'city'] = 'MISSION'
kansas_data.loc[kansas_data['city'].str.contains('MISSSION'),'city'] = 'MISSION'
kansas_data.loc[kansas_data['city'].str.contains('PRAIRE VILLAGE'),'city'] = 'PRAIRIE VILLAGE'
kansas_data.loc[kansas_data['city'].str.contains('WESTWOOD_HILLS'),'city'] = 'WESTWOOD'
kansas_data.loc[kansas_data['city'].str.contains('NEW CENTRY'),'city'] = 'NEW CENTURY'
kansas_data.loc[kansas_data['city'].str.contains('BONNER SPRING'),'city'] = 'BONNER SPRINGS'
kansas_data.loc[kansas_data['city'].str.contains('MERIAM'),'city'] = 'MERRIAM'
kansas_data.loc[kansas_data['city'].str.contains('OLA'),'city'] = 'OLATHE'
kansas_data.loc[kansas_data['city'].str.contains('OVELAND PARK'),'city'] = 'OVERLAND PARK'
kansas_data.loc[kansas_data['city'].str.contains('STILLWELL'),'city'] = 'STILWELL'
kansas_data.loc[kansas_data['city'].str.contains('ROELAND'),'city'] = 'OVERLAND PARK'
kansas_data.loc[kansas_data['city'].str.contains('PRAIRIE'),'city'] = 'PRAIRIE VILLAGE'
kansas_data.loc[kansas_data['city'].str.contains('PRARIE VILLAGE'),'city']='PRAIRIE VILLAGE'
kansas_data.loc[kansas_data['city'].str.contains('NEW_CENTURY'),'city'] = 'NEW CENTURY'
kansas_data.loc[kansas_data['city'].str.contains('MISSION_HILLS'),'city'] = 'MISSION HILLS'
kansas_data.loc[kansas_data['city'].str.contains('SPRING_HILLS'),'city'] = 'SPRING HILLS'
kansas_data.loc[kansas_data['city'].str.contains(' OVPK'),'city'] = 'OVERLAND PARK'
kansas_data.loc[kansas_data['city'].str.contains('MISISON'),'city'] = 'MISSION'
kansas_data.loc[kansas_data['city'].str.contains('DESOTO'),'city'] = 'DE SOTO'
kansas_data.loc[kansas_data['city'].str.contains('DE SOTO'),'city'] = 'DE SOTO'
kansas_data.loc[kansas_data['city'].str.contains('DSTO'),'city'] = 'DE SOTO'
kansas_data.loc[kansas_data['city'].str.contains('MERRIAM KS'),'city'] = 'MERRIAM'
kansas_data.loc[kansas_data['city'].str.contains('SPRING_HILLS'),'city'] = 'SPRING HILLS'
kansas_data.loc[kansas_data['city'].str.contains('SHWN'),'city'] = 'SHAWNEE'
kansas_data.loc[kansas_data['city'].str.contains('MISSON_WOODS'),'city'] = 'MISSION WOODS'


In [138]:
kansas_data=kansas_data[kansas_data.city!='20705 W. 151ST STREE']
kansas_data=kansas_data[kansas_data.city!='WAUKESHA']

In [139]:
#kansas_data.city.value_counts()

## Finally try to see whether there's anything to be done on the x, y coordinates.

In [140]:
kansas_data['x'].describe()

count    335611.000000
mean        -93.656092
std          11.963704
min         -97.610003
25%         -94.791173
50%         -94.719215
75%         -94.670748
max          39.051429
Name: x, dtype: float64

In [141]:
x_cord = kansas_data['x']

In [142]:
x_cord.loc[x_cord < -97].count()

2

In [143]:
x_cord.loc[x_cord < -96].count()

2

In [144]:
x_cord.loc[x_cord < -95].count()

2435

In [145]:
x_cord.loc[x_cord < -95.05].count()

87

In [146]:
x_cord.loc[x_cord < -95.1].count()

2

In [147]:
kansas_data = kansas_data[kansas_data['x'] >= -95.1]

In [148]:
x_cord.loc[x_cord > -94].count()    ## 2707 wrong x cordinates

2707

In [149]:
x_cord.loc[x_cord > -94].value_counts()

39.011965    143
39.023860     78
39.025393     49
39.010610     27
38.994423     21
39.013538     21
39.010517     16
39.015943     16
39.022364     13
38.992380     12
39.006356     12
39.016287     10
39.016030      9
39.011932      8
39.023055      8
39.029518      7
39.040382      7
39.011817      7
39.021475      7
39.008255      7
39.012736      7
39.024287      7
39.013670      7
39.016917      6
39.012458      6
39.013915      6
38.988318      5
39.012841      5
39.014303      5
39.010101      5
            ... 
39.008788      1
39.010403      1
39.001943      1
38.995649      1
38.988601      1
39.012684      1
39.008045      1
39.042724      1
39.004149      1
39.022248      1
38.994575      1
39.002782      1
38.997239      1
39.013031      1
39.026846      1
39.005318      1
39.035070      1
39.022355      1
39.032762      1
39.036740      1
39.019798      1
39.051171      1
39.040316      1
38.999947      1
38.991890      1
38.997174      1
39.001962      1
39.012106     

In [150]:
kansas_data.loc[kansas_data['x'] == 39.051429]

,id,agency,daterept,hourrept,ucrcode,offense,neighborhood,subdivision,spx,spy,...,Minute,Time,Date,Day,DayOfWeek,Week,Month,Year,FullDayOfWeek,FullMonth
334976,3002069,SPD,8/24/15 0:00,15:46:00,23,"THEFT OF PROPERTY OR SERVICES; VALUE $1,000 TO...",NaN,NaN,2205067.0,277036.0,...,46,946,2015-08-24,24,0,35,8,2015,Monday,Aug


In [151]:
kansas_data.loc[kansas_data['x'] == 39.051429]['y']

334976   -94.855833
Name: y, dtype: float64

### We see there's a problem of wrong input of x and y. It seems there are 2707 cases where people mistake x for y and mistake y for x.

In [152]:
kansas_data['y'].describe()

count    335609.000000
mean         37.846984
std          11.957851
min         -94.897892
25%          38.882506
50%          38.917891
75%          38.969716
max          39.369999
Name: y, dtype: float64

In [153]:
y_cord = kansas_data['y']

In [154]:
y_cord.loc[y_cord < 38].count()   ## 2707 wrong y coordinates

2707

In [155]:
y_cord.loc[y_cord < 38].value_counts()   ## 2707 wrong y coordinates

-94.772829    143
-94.782270     78
-94.849959     49
-94.767982     27
-94.717735     21
-94.713525     21
-94.765834     16
-94.743511     16
-94.724791     13
-94.767166     12
-94.705533     12
-94.725471     10
-94.738690     10
-94.734575      8
-94.775383      8
-94.722023      7
-94.713874      7
-94.744538      7
-94.720092      7
-94.720095      7
-94.825173      7
-94.730274      7
-94.749485      6
-94.705624      6
-94.826514      6
-94.721454      5
-94.713249      5
-94.743221      5
-94.844000      5
-94.710278      5
             ... 
-94.747169      1
-94.847170      1
-94.783102      1
-94.716898      1
-94.748457      1
-94.713305      1
-94.712113      1
-94.731525      1
-94.746443      1
-94.865623      1
-94.830402      1
-94.748885      1
-94.763644      1
-94.860345      1
-94.839055      1
-94.717796      1
-94.711776      1
-94.747571      1
-94.708703      1
-94.734399      1
-94.755927      1
-94.750083      1
-94.850010      1
-94.762765      1
-94.714960

### We want to make sure that the wrong input of x corresponds to the wrong input of y.

In [156]:
kansas_data[kansas_data['x'] > -94]['y'].describe()

count    2707.000000
mean      -94.758371
std         0.048858
min       -94.897892
25%       -94.780556
50%       -94.744538
75%       -94.717787
max       -94.624417
Name: y, dtype: float64

In [157]:
kansas_data[kansas_data['y'] < 38]['x'].describe()

count    2707.000000
mean       39.013336
std         0.016459
min        38.741549
25%        39.004136
50%        39.012503
75%        39.023860
max        39.051429
Name: x, dtype: float64

### Now what we need to do is we need to switch x and y coordinates for those pieces of data.

In [158]:
wrong_y_cord = kansas_data[kansas_data['x'] > - 94]['y']

In [159]:
#wrong_y_cord

In [160]:
wrong_y_cord.index

Int64Index([332907, 332908, 332909, 332910, 332911, 332912, 332913, 332914,
            332915, 332916,
            ...
            335604, 335605, 335606, 335607, 335608, 335609, 335610, 335611,
            335612, 335613],
           dtype='int64', length=2707)

In [161]:
wrong_x_cord = kansas_data[kansas_data['y'] < 38]['x']

In [162]:
wrong_x_cord.index

Int64Index([332907, 332908, 332909, 332910, 332911, 332912, 332913, 332914,
            332915, 332916,
            ...
            335604, 335605, 335606, 335607, 335608, 335609, 335610, 335611,
            335612, 335613],
           dtype='int64', length=2707)

In [163]:
sum(wrong_x_cord.index != wrong_y_cord.index)

0

In [164]:
kansas_data.loc[wrong_x_cord.index, 'x'] = wrong_y_cord
kansas_data.loc[wrong_y_cord.index, 'y'] = wrong_x_cord

In [165]:
kansas_data['x'].describe()

count    335609.000000
mean        -94.735063
std           0.079062
min         -95.073315
25%         -94.791439
50%         -94.719921
75%         -94.671605
max         -94.596676
Name: x, dtype: float64

In [166]:
kansas_data['y'].describe()

count    335609.000000
mean         38.925978
std           0.060539
min          38.569660
25%          38.883036
50%          38.919071
75%          38.970980
max          39.369999
Name: y, dtype: float64

In [167]:
kansas_data['x'].loc[kansas_data['x'] < -95].count()

2433

In [168]:
#missing_count(kansas_data)

In [169]:
kansas_data = kansas_data.dropna(subset = ['ucrcode'])

In [170]:
missing_count(kansas_data)['ucrcode']

0

In [171]:
kansas_data.to_csv('kansas_data_cleaned.csv')

In [172]:
kansas_data.shape

(335484, 35)

### We've reduced the dataset from 335614 cases to 335484 cases, and we cleaned the data and add more features.